In [1]:
import os
import pandas as pd
import re
import datetime
from tqdm import tqdm

In [2]:
import time

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Время выполнения функции '{func.__name__}': {execution_time:.6f} секунд")
        print(f"Аргументы: args={args}, kwargs={kwargs}")
        return result
    return wrapper

In [3]:
def filter_vouts(vout, check):
    tmp = pd.merge(left=vout, right=check, how='inner', on='date')
    tmp['delta_time'] = (tmp['time'] - tmp['ltime']) / datetime.timedelta(minutes=1)
    tmp = tmp[tmp['delta_time'].abs() < 30]
    tmp = tmp[tmp['delta_time'] > 0]
    tmp['delta_sum'] = 2*(tmp['value'] - tmp['btcsum']) / (tmp['btcsum'] + tmp['value'])
    tmp = tmp[tmp['delta_sum'] < 0.1]
    tmp = tmp[tmp['delta_sum'] > 0]
    return tmp

In [4]:
import json

def decode_v(x):
    r = x[1:-1]
    r = r.replace("'",'"')
    s = r.split(': ')[0]+': '
    r = r.split(s)[1:]
    r = [s+i for i in r]
    r = [i[:-2] if i[-2:]==', ' else i for i in r]
    r = [json.loads(i) for i in r]
    return r

In [5]:
def get_vout(block):
    block = block.copy()
    block.loc[:, 'vout'] = block['vout'].map(decode_v)

    t = enumerate(block['vout'].values)
    t = [{**{'index': i}, **j} for (i, v) in t for j in v]
    t = pd.DataFrame.from_dict(t)

    vout = t['scriptPubKey'].values
    vout = pd.DataFrame.from_records(vout)
    vout = pd.concat([t.iloc[:, :2], vout], axis=1)
    vout = pd.merge(vout, block.reset_index(), how='left', on='index')
    vout = vout.dropna()
    vout.loc[:, 'time'] = pd.to_datetime(vout['time'].map(ts2dt))
    vout.loc[:, 'date'] = vout['time'].dt.date
    return vout

In [6]:
from tqdm import tqdm

def ts2dt(x):
    return datetime.datetime.utcfromtimestamp(x)

@timing_decorator
def filter_df(data_path):
    pattern = r'.*/script_rpc_blocks_(\d+)_(\d+)\.csv'
    match = re.match(pattern, data_path)
    if match:
        start_block = int(match.group(1))
        end_block = int(match.group(2))
        new_df_name = '{}/filtered_rpc_vouts_{}_{}.csv'.format(data_path.rsplit('/', 1)[0], {}, {})
        df = pd.read_csv(data_path).sort_values(by='time')
        # df['rtime'] = df['time'].map(ts2dt)
        len_old = len(df)
        print(f"Загружен датафрейм {data_path}")
        print(f"Длина датафрейма {len_old}")
        # check = pd.read_csv('./check_btc.csv', index_col=0)
        # check['ltime'] = pd.to_datetime(check['ltime'])
        # check = check.sort_values(by='ltime')
        # df = pd.merge_asof(df, check, left_on='rtime', right_on='ltime', direction='nearest')
        # df = df[df['ltime'] - df['rtime'] < datetime.timedelta(minutes=30)]
        # len_new = len(df)
        # print(f"Новая длина {len_new}. Отфильтровано по времени {len_old - len_new}")

        check = pd.read_csv('./check_btc.csv', index_col=0)
        check['ltime'] = pd.to_datetime(check['ltime'])
        check = check.sort_values(by='ltime')
        check['date'] = check['ltime'].dt.date

        chunk_size = 1000

        result = []

        for start in tqdm(range(0, len(df), chunk_size), desc="Processing chunks"):
            chunk = df.iloc[start:start + chunk_size]
            # print(f"Parsing chank {start}")
            vout = get_vout(chunk)
            res = filter_vouts(vout, check)
            result.append(res)

        df = pd.concat(result, ignore_index=True)
        print(f"Длина результата : {len(df)}")
        df.to_csv(new_df_name.format(start_block, end_block), index=False)


In [65]:
data_path = "part1/script_rpc_blocks_834577_834660.csv"

In [66]:
filter_df(data_path)

Загружен датафрейм part1/script_rpc_blocks_834577_834660.csv
Длина датафрейма 175078


Processing chunks: 100%|██████████| 176/176 [00:15<00:00, 11.43it/s]


Длина результата : 37
Время выполнения функции 'filter_df': 28.865072 секунд
Аргументы: args=('part1/script_rpc_blocks_834577_834660.csv',), kwargs={}


In [67]:
filter_df("part1/script_rpc_blocks_834661_834743.csv")

Загружен датафрейм part1/script_rpc_blocks_834661_834743.csv
Длина датафрейма 205197


Processing chunks: 100%|██████████| 206/206 [00:20<00:00, 10.07it/s]


Длина результата : 0
Время выполнения функции 'filter_df': 33.521261 секунд
Аргументы: args=('part1/script_rpc_blocks_834661_834743.csv',), kwargs={}


In [7]:
filter_df("part2/script_rpc_blocks_834744_834896.csv")

Загружен датафрейм part2/script_rpc_blocks_834744_834896.csv
Длина датафрейма 65322


Processing chunks: 100%|██████████| 66/66 [00:10<00:00,  6.43it/s]

Длина результата : 0
Время выполнения функции 'filter_df': 18.853105 секунд
Аргументы: args=('part2/script_rpc_blocks_834744_834896.csv',), kwargs={}


In [8]:
filter_df("part2/script_rpc_blocks_834897_835048.csv")

Загружен датафрейм part2/script_rpc_blocks_834897_835048.csv
Длина датафрейма 320838


Processing chunks: 100%|██████████| 321/321 [00:41<00:00,  7.76it/s]


Длина результата : 23
Время выполнения функции 'filter_df': 79.658086 секунд
Аргументы: args=('part2/script_rpc_blocks_834897_835048.csv',), kwargs={}


In [9]:
filter_df("part3/script_rpc_blocks_835049_835180.csv")

Загружен датафрейм part3/script_rpc_blocks_835049_835180.csv
Длина датафрейма 333322


Processing chunks: 100%|██████████| 334/334 [00:42<00:00,  7.91it/s]


Длина результата : 18
Время выполнения функции 'filter_df': 74.948828 секунд
Аргументы: args=('part3/script_rpc_blocks_835049_835180.csv',), kwargs={}


In [10]:
data_frames = [
    pd.read_csv("part1/filtered_rpc_vouts_834577_834660.csv"),
    pd.read_csv("part2/filtered_rpc_vouts_834897_835048.csv"),
    pd.read_csv("part3/filtered_rpc_vouts_835049_835180.csv")
]
resdf = pd.concat(data_frames, ignore_index=True)

In [12]:
resdf.to_csv("filtered_rpc_vouts.csv")

In [ ]:
df = pd.read_csv(data_path)

In [26]:
df2 = df.head(10000)

In [37]:
txs = df2
blocks = sorted(txs['blockNumber'].unique())


In [38]:
check = pd.read_csv('./check_btc.csv', index_col=0)
check['ltime'] = pd.to_datetime(check['ltime'])
check = check.sort_values(by='ltime')
check['date'] = check['ltime'].dt.date

In [41]:
result = []
for bn in blocks:
    print(f"Parsing block {bn}")
    block = txs[txs['blockNumber'] == bn]
    vout = get_vout(block)
    res = filter_vouts(vout, check)
    result.append(res)

Parsing block 834577
Parsing block 834578
Parsing block 834579
Parsing block 834580
Parsing block 834581


In [42]:
pd.concat(result, ignore_index=True)

,index,value,asm,desc,hex_x,address,type,in_active_chain,txid,hash,...,blockhash,confirmations,time,blocktime,blockNumber,date,ltime,btcsum,delta_time,delta_sum
0,69,0.000447,OP_HASH160 78f607fec1a3974a5adb0587b319acda5c7...,addr(3Cibk7WTryDWNAPDdDRYwPCF6GTQLiHRZb)#vxsapr9t,a91478f607fec1a3974a5adb0587b319acda5c72c18987,3Cibk7WTryDWNAPDdDRYwPCF6GTQLiHRZb,scripthash,True,51b6c9d1f8a5f0713985a2a70a86a68d20233cc9fec37d...,9ad76c168921ad0a7864d03e35bb3af43c4e1132c7e868...,...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10923.0,2024-03-13 22:40:42,1.710370e+09,834577.0,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.011080
1,79,0.000453,OP_DUP OP_HASH160 2a46b6273dd8aceeee06643c9afb...,addr(14rY6BUNhGFKek5dEzTpi7X2KgWCiocebx)#0nvhhx29,76a9142a46b6273dd8aceeee06643c9afbf33af27688f8...,14rY6BUNhGFKek5dEzTpi7X2KgWCiocebx,pubkeyhash,True,704e5a2192c4d214150bc848cb89ecba7ad8a6930b9d3d...,704e5a2192c4d214150bc848cb89ecba7ad8a6930b9d3d...,...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10923.0,2024-03-13 22:40:42,1.710370e+09,834577.0,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.023495
2,117,0.000473,0 568daefc6315f737fddafcb4fed275195413c29f,addr(bc1q26x6alrrzhmn0lw6lj60a5n4r92p8s5l9p9zd...,0014568daefc6315f737fddafcb4fed275195413c29f,bc1q26x6alrrzhmn0lw6lj60a5n4r92p8s5l9p9zd8,witness_v0_keyhash,True,c9aece470ac1093c6e534df10f17f74b082dddd0ec7aa0...,81b7ecb7a82f0158820903d186cb71536570054fed6fec...,...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10923.0,2024-03-13 22:40:42,1.710370e+09,834577.0,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.066760
3,136,0.000459,OP_HASH160 267e4648ceef77f9872d6f1d2a5530c4b0c...,addr(35CYsYS4qLL7TXVrS4utzT1XEV1KwpJoh9)#x7g3508n,a914267e4648ceef77f9872d6f1d2a5530c4b0ce174987,35CYsYS4qLL7TXVrS4utzT1XEV1KwpJoh9,scripthash,True,44279177cf9c365d02b3cb60517bacf40b47604d4bc8a8...,f4be0018273a6ae9ab6848dafc064b81918ba254f3cd2f...,...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10923.0,2024-03-13 22:40:42,1.710370e+09,834577.0,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.036476
4,143,0.000477,0 44bbe4de0d96b86436b89b26dd99e4d6fb6ffa49,addr(bc1qgja7fhsdj6uxgd4cnvndmx0y6makl7jfm626e...,001444bbe4de0d96b86436b89b26dd99e4d6fb6ffa49,bc1qgja7fhsdj6uxgd4cnvndmx0y6makl7jfm626ea,witness_v0_keyhash,True,688a2c98a8597d031d16880ed7d205a6876efadd54fecd...,3be3a86788e17d4d3a4537b380d33cde15d88c33ced968...,...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10923.0,2024-03-13 22:40:42,1.710370e+09,834577.0,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.075504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,2749,0.002576,OP_HASH160 f5ff6b0a842825258c4a8ecb85a25d35c93...,addr(3Q7jTJe6JmxxUVdppCfmDA4p9bBhFgoUcQ)#4fvndnag,a914f5ff6b0a842825258c4a8ecb85a25d35c93743fc87,3Q7jTJe6JmxxUVdppCfmDA4p9bBhFgoUcQ,scripthash,True,a46cc3d8d033d5e5e8a5b94b9a8effe7565dde1c9d1f4b...,d26d4475de830d88209229ff45055e392c78e0215e3b01...,...,0000000000000000000028709c9e20ff32060ed5721bda...,10922.0,2024-03-13 23:09:15,1.710371e+09,834578.0,2024-03-13,2024-03-13 22:59:40,0.002550,9.583333,0.009995
179,2781,0.002745,OP_DUP OP_HASH160 74552f3d7e5819dc923047c98261...,addr(1Bc7RyguMRpVV5iwj5Ej9J7e6KH25yakWg)#4zrlxd5e,76a91474552f3d7e5819dc923047c98261ca9be60f5bd6...,1Bc7RyguMRpVV5iwj5Ej9J7e6KH25yakWg,pubkeyhash,True,2f61b750bfe386aac549a1c5bd47ef59909cdbc1baf3d1...,0b33de95ec2bcf227f5507cf92a126ee66d4568d6b9d61...,...,0000000000000000000028709c9e20ff32060ed5721bda...,10922.0,2024-03-13 23:09:15,1.710371e+09,834578.0,2024-03-13,2024-03-13 22:59:40,0.002550,9.583333,0.073816
180,2820,0.002671,0 274f988984fa36d7fd971aee4a06be527aa97115,addr(bc1qya8e3zvylgmd0lvhrthy5p472fa2jug4cjpqy...,0014274f988984fa36d7fd971aee4a06be527aa97115,bc1qya8e3zvylgmd0lvhrthy5p472fa2jug4cjpqyn,witness_v0_keyhash,True,1558be51938ffb2ffbd0773eff69c53ae95755c3328a6a...,8bb0b750aa6e561df2abe8f8a5d9767fdd9adf95f9233c...,...,0000000000000000000028709c9e20ff32060ed5721bda...,10922.0,2024-03-13 23:09:

In [14]:
tmp = filter_vouts(vout, check)

In [15]:
tmp

{'index': {30247: 69,
  33458: 79,
  50189: 117,
  54583: 136,
  72497: 143,
  86693: 163,
  105790: 221,
  134013: 304,
  159363: 358,
  190628: 413,
  203810: 454,
  218006: 459,
  250623: 529,
  258228: 545,
  259073: 545,
  261439: 545,
  268875: 545,
  277832: 545,
  278001: 545,
  283240: 545,
  284761: 545,
  364698: 770,
  396639: 912,
  400019: 925,
  415736: 926,
  469478: 1110,
  471168: 1117},
 'value': {30247: 0.00044741,
  33458: 0.000453,
  50189: 0.00047304,
  54583: 0.00045892,
  72497: 0.0004772,
  86693: 0.00047192,
  105790: 0.00045,
  134013: 0.00048248,
  159363: 0.00047272,
  190628: 0.00045208,
  203810: 0.00045386,
  218006: 0.00045,
  250623: 0.00047897,
  258228: 0.00048218,
  259073: 0.00048233,
  261439: 0.00047907,
  268875: 0.00047923,
  277832: 0.00046749,
  278001: 0.00046899,
  283240: 0.00048118,
  284761: 0.00048235,
  364698: 0.00048492,
  396639: 0.00048066,
  400019: 0.00048,
  415736: 0.00047725,
  469478: 0.00044689,
  471168: 0.00047016},
 'asm

In [33]:
df2['blockNumber'].unique()

array([834577, 834578, 834579, 834580, 834581])

In [22]:
decode_v(df['vout'][3])

[{'value': 0.0017763,
  'n': 0,
  'scriptPubKey': {'asm': '0 21c4c391276813e2ef891fe1f6604be6fa2c7aa9',
   'desc': 'addr(bc1qy8zv8yf8dqf79mufrlslvcztumazc74fjps77t)#vsj92tpd',
   'hex': '001421c4c391276813e2ef891fe1f6604be6fa2c7aa9',
   'address': 'bc1qy8zv8yf8dqf79mufrlslvcztumazc74fjps77t',
   'type': 'witness_v0_keyhash'}}]

In [18]:
len(df)

10000

In [19]:
check = pd.read_csv('./check_btc.csv', index_col=0)

In [9]:
check['ltime'] = pd.to_datetime(check['ltime'])

In [10]:
check = check.sort_values(by='ltime')

In [11]:
len(check)

4300

In [12]:
check

,ltime,btcsum
116250,2024-02-29 21:06:53,0.000883
83376,2024-02-29 21:07:09,0.001767
165801,2024-02-29 21:13:19,0.000353
116267,2024-02-29 21:19:29,0.000883
132860,2024-02-29 21:41:50,0.000447
...,...,...
66759,2024-03-31 19:24:05,0.000153
66736,2024-03-31 19:25:06,0.000611
66744,2024-03-31 19:31:22,0.000458
66738,2024-03-31 19:31:37,0.000611


In [121]:
df['rtime'] = df['time'].map(ts2dt)
df = df.sort_values(by='rtime')

In [13]:
ind = 300
check['ltime'][ind:ind + 20]

412834   2024-03-02 05:30:40
363499   2024-03-02 05:31:43
248359   2024-03-02 05:31:57
511273   2024-03-02 05:32:06
363535   2024-03-02 05:46:47
330600   2024-03-02 05:46:52
511282   2024-03-02 05:47:57
264895   2024-03-02 05:47:58
396420   2024-03-02 05:48:21
248470   2024-03-02 05:48:59
511365   2024-03-02 05:49:06
330546   2024-03-02 05:50:43
281313   2024-03-02 05:50:57
379964   2024-03-02 05:52:14
330620   2024-03-02 05:52:27
396360   2024-03-02 05:54:46
297764   2024-03-02 05:54:47
396419   2024-03-02 05:56:29
281336   2024-03-02 05:59:09
248467   2024-03-02 06:05:03
Name: ltime, dtype: datetime64[ns]

In [123]:
df['rtime'][:10]

0      2024-03-14 12:14:26
1732   2024-03-14 12:14:26
1733   2024-03-14 12:14:26
1734   2024-03-14 12:14:26
1735   2024-03-14 12:14:26
1736   2024-03-14 12:14:26
1737   2024-03-14 12:14:26
1738   2024-03-14 12:14:26
1731   2024-03-14 12:14:26
1739   2024-03-14 12:14:26
Name: rtime, dtype: datetime64[ns]

In [124]:
import datetime
def ts2dt(x):
    return datetime.datetime.utcfromtimestamp(x)

In [127]:
df = pd.merge_asof(df, check, left_on='rtime', right_on='ltime', direction='nearest')

In [128]:
# df['rtime'] = pd.merge_asof(df, check, left_on='ltime', right_on='ltime', direction='nearest')

In [129]:
df.head()

,in_active_chain,txid,hash,version,size,vsize,weight,locktime,vin,vout,hex,blockhash,confirmations,time,blocktime,blockNumber,rtime,ltime,btcsum
0,True,cc3d2ae9ed7f719dccc5475638ace7cfce03af7376f573...,698c5a5b76a9a58599060741559bc898ab5b585dee3791...,1,396,369,1476,0,[{'coinbase': '0365bc0c1d506f77657265642062792...,"[{'value': 5.46e-06, 'n': 0, 'scriptPubKey': {...",0100000000010100000000000000000000000000000000...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601
1,True,08e994436242dd2206abae88518f169d62d1ed3fe6c17f...,bce4defb2a436d4be7197a952c10b4e9ce2edc039c53f4...,1,223,141,562,0,[{'txid': 'c8e8671a2abf065f365ad7de9eca8b88b94...,"[{'value': 0.00042065, 'n': 0, 'scriptPubKey':...",010000000001011513483e594fdf4985f85bab9da840b9...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601
2,True,d4215e88e7ea22aa6167469973436999bd86d9e1f1fad7...,cae31fab4c872abb4a08882aedd09d4184939eb8f3f036...,1,520,277,1105,0,[{'txid': '5504708a3e0484f27790808ef4054ab115e...,"[{'value': 0.01249804, 'n': 0, 'scriptPubKey':...",01000000000103e5dbb73a99683b8b02cac55c9508e415...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601
3,True,38e14aa2f30b0114d5b010c6457b18531d035fa450b0c4...,020be95be7695aeaa03fd405996b47412ee796c3a39fa5...,1,193,111,442,0,[{'txid': '08512c45c4a4dc6fc3267394fc085593764...,"[{'value': 0.02707927, 'n': 0, 'scriptPubKey':...",01000000000101350e2752c2d682c74fe977d7dab34076...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601
4,True,5e6e893616b08a16d690d615ba7048c404855828f7c789...,28509bbcbce90dcddaa296426f64de151ec55ebbf7e1e4...,1,404,213,851,0,[{'txid': '2716ba171d41efdd949218de1dcf87f7b18...,"[{'value': 0.00247116, 'n': 0, 'scriptPubKey':...",010000000001013d2b4bb50189eefe290dcb85e74180b1...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601


In [130]:
check.head(40)

,ltime,btcsum
116250,2024-02-29 21:06:53,0.000883
83376,2024-02-29 21:07:09,0.001767
165801,2024-02-29 21:13:19,0.000353
116267,2024-02-29 21:19:29,0.000883
132860,2024-02-29 21:41:50,0.000447
66573,2024-02-29 21:55:23,0.003579
66955,2024-02-29 21:56:12,0.003042
149396,2024-02-29 22:00:57,0.000358
66562,2024-02-29 22:10:34,0.004474
593959,2024-02-29 22:54:32,0.000184


In [52]:
df['ltime']

0      2024-03-14 11:36:48
1732   2024-03-14 11:36:48
1733   2024-03-14 11:36:48
1734   2024-03-14 11:36:48
1735   2024-03-14 11:36:48
               ...        
8820   2024-03-14 12:55:34
8819   2024-03-14 12:55:34
8818   2024-03-14 12:55:34
8829   2024-03-14 12:55:34
9999   2024-03-14 12:55:34
Name: ltime, Length: 10000, dtype: datetime64[ns]

In [135]:
df = df[df['ltime'] - df['rtime'] < datetime.timedelta(minutes=1)]

In [136]:
len(df)

4315

In [13]:
(df['ltime'] - df['time'])[0]

Timedelta('-1 days +23:54:43')

In [19]:
x = df['time'][0]
x

Timestamp('2024-03-13 22:40:42')

In [20]:
y = df['ltime'][0]
y

Timestamp('2024-03-13 22:35:25')

In [27]:
(y - x) < datetime.timedelta(minutes=30)

True